Setting working directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/DATA309 Capstone Project - Lego4/write ups/Additional files/

# Retrieving and processing (sentiment analysis) of brickset review data

The code below was used to retrieve the reviews form every set on brickset, the start and end indexs were updated accordingly as each chunk was complete (note that this code is in r, not python)

In [ ]:
allreviewdata <- data.frame()
broken_reviews <- c('306')

start_index <- 14001
end_index <- 19409

# Loop through the sets
for (i in start_index:end_index) {
  if (legosets$reviewCount[i] > 0 & !(i %in% broken_reviews)) {

    setid <- legosets$setID[i]

    setreviewdata <- getReviews(setid)

    setreviewdata$setID <- rep(setid, nrow(setreviewdata))

    allreviewdata <- rbind(allreviewdata, setreviewdata)
  }
}

# View the combined review data
print(allreviewdata)

write.csv(allreviewdata, file = f"{start_index}z{end_index}reviews.csv", row.names = TRUE)

The following code combined all perviously collected review data, then combining and exporting this all as one file

In [ ]:
import csv

review_files = ["1z2000reviews.csv", "2001z4000reviews.csv", "4001z6000reviews.csv", "6001z10000reviews.csv", "10001z14000reviews.csv", "14001z19409reviews.csv"]

all_data = []
for i in range(len(review_files)):
  file_to_process = review_files[i]
  with open(file_to_process, newline='') as csvfile:
    reader = csv.reader(csvfile)
    if i == 0:
      for row in reader:
        all_data.append(row)
    else:
      next(reader)
      for row in reader:
        all_data.append(row)

import pandas as pd

newdata = pd.DataFrame(all_data[1:], columns=all_data[0])

newdata.to_csv("all_reviews_combined.csv", index=False)

Inialise sentiment analysis

In [ ]:
file_to_process = "all_reviews_combined.csv" #NEED TO CHANGE THIS DEPENDING ON WHAT YOU WANNA UPDATE
data_list = []
with open(file_to_process, newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        data_list.append(row)

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

Complete sentiment analysis using nltk on all brickset reviews and exporting it

In [ ]:
data_list[0] = data_list[0][:12]
title_headers = ["title_neg", "title_neu", "title_pos", "title_comp"]
review_headers = ["review_neg", "review_neu", "review_pos", "review_comp"]
data_list[0] += title_headers + review_headers
data_list[0] = data_list[0][:20]

for i in range(1, len(data_list)):
  data_list[i] = data_list[i][:12]
  ts = sia.polarity_scores(data_list[i][3])
  rs = sia.polarity_scores(data_list[i][4])
  ts_list = [ts['neg'], ts['neu'], ts['pos'], ts['compound']]
  rs_list = [rs['neg'], rs['neu'], rs['pos'], rs['compound']]
  data_list[i] += ts_list + rs_list
  data_list[i] = data_list[i][:20]


newdata = pd.DataFrame(data_list[1:], columns=data_list[0])

newdata.to_csv("brickset all reviews", index=False)

# Adding sentiment analysis back to brickset dataset

In [ ]:
import csv

file_to_process1 = "brickset all reviews.csv"
file_to_process2 = "brickset legoset data.csv"
all_reviews_data = []
brickset_data = []
with open(file_to_process1, newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        all_reviews_data.append(row)
with open(file_to_process2, newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        brickset_data.append(row)

Making missing data easier to handle

In [ ]:
for i in range(1, len(brickset_data)):
  for j in range(1, len(brickset_data[i])):
    if brickset_data[i][j] == 'NA':
      brickset_data[i][j] = None

for i in range(1, len(all_reviews_data)):
  for j in range(6, 11):
    if all_reviews_data[i][j] == '0':
      all_reviews_data[i][j] = None

Finding reviews per set

In [ ]:
review_dict = {}
review_dict['setID'] = all_reviews_data[0][6:11] + all_reviews_data[0][12:]
for i in range(1, len(all_reviews_data)):
  setID = all_reviews_data[i][11]
  review_dict[setID] = review_dict.get(setID, []) + [all_reviews_data[i][6:11] + all_reviews_data[i][12:]]

Finding the average sentiment scores of each lego sets reviews and then exporting this as the new usable brickset dataset

In [ ]:
review_avgs = [["setID", "num_reviews"] + [review_dict['setID'][i] for i in range(13)]]  # Assuming 13 columns from review_dict

fill_in = [None] * 13  # Adjusting the fill_in to match the number of columns

def avg_ith_column_noskip(data, i):
    total = 0
    count = 0
    for entry in data:
        total += float(entry[i])
        count += 1
    return total / count

def avg_ith_column_skips(data, i):
    total = 0
    count = 0
    for entry in data:
        if entry[i] is not None:
          total += float(entry[i])
          count += 1
    if count == 0:
        return None
    else:
      return total / count

for i in range(1, len(brickset_data)):
    setID = brickset_data[i][0]
    count = len(review_dict.get(setID, []))
    row = [setID, count]

    if count > 0:
        for j in range(5):
            result = avg_ith_column_skips(review_dict.get(setID, []), j)
            row.append(result)
        for j in range(5, 13):
            result = avg_ith_column_noskip(review_dict.get(setID, []), j)
            row.append(result)
        review_avgs.append(row)
    else:
        review_avgs.append(row + fill_in)

import pandas as pd

newdata1 = pd.DataFrame(review_avgs[1:], columns=review_avgs[0])

newdata1.to_csv("usable_review_data.csv", index=False)

newdata2 = pd.DataFrame(brickset_data[1:], columns=brickset_data[0])

newdata2.to_csv("usable_brickset_data.csv", index=False)

# Handling retail and market price data

Exploring availability of retail price data

In [ ]:
US_Price = set()
UK_Price = set()
CA_Price = set()
DE_Price = set()
all_prices = set()

for i in range(1, len(brickset_data)):
    for j in [20, 23, 26, 29]:
      if brickset_data[i][j] != '':
        if j == 20:
          US_Price.add(brickset_data[i][1])
        if j == 23:
          UK_Price.add(brickset_data[i][1])
        if j == 26:
          CA_Price.add(brickset_data[i][1])
        if j == 29:
          DE_Price.add(brickset_data[i][1])
        all_prices.add(brickset_data[i][1])

print(len(US_Price), len(UK_Price), len(CA_Price), len(DE_Price))
print(len(all_prices))

US_all_difference = all_prices.difference(US_Price)
print("")
print(len(US_all_difference))
print(len(UK_Price.intersection(US_all_difference)), len(CA_Price.intersection(US_all_difference)),len(DE_Price.intersection(US_all_difference)))